In [1]:
import numpy as np
import random

In [2]:
class Player():
    def __init__(self, name):
        
        self.name = name
        self.cards = None
    
    ### -------------------------------------------------------------
    
    ### TO BE IMPLEMENTED - player's strategy 
    ### input: declared card, i.e., the card which is supposed
    ### to be the top card of the pile: If None - you can put any card you want because
    ### (a) it is the first turn (pile is empty) or (b) some cards were drawn in the previous turn)
    ### output: - player's true decision, player's declaration (if not equal - (s)he cheats)
    
    def putCard(self, declared_card):
        ### DO NOT REMOVE TRUE CARD cards.remove!!!
        ### return an object (not id): self.cards[id], not id
        ### for instance: return self.cards[0], self.cards[0] 
        ### IMPORTANT: If you want to draw cards instead of put, return "draw"
        ### for instance: return "draw" 
        return self.cards[0], self.cards[0] 
    
    ### TO BE IMPLEMENTED - Decide whether to check or not opponent's move (return True or False)
    def checkCard(self, opponent_declaration):
        pass
    
    ### Notification sent at the end of a round
    ### One may implement this method, capture data, and use it to get extra info
    ### -- checked = TRUE -> someone checked. If FALSE, the remaining inputs do not play any role
    ### -- iChecked = TRUE -> I decided to check my opponent (so it was my turn); 
    ###               FALSE -> my opponent checked and it was his turn
    ### -- iDrewCards = TRUE -> I drew cards (so I checked but was wrong or my opponent checked and was right); 
    ###                 FALSE -> otherwise
    ### -- revealedCard - some card (X, Y). Only if I checked.
    ### -- noTakenCards - number of taken cards
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    ### -------------------------------------------------------------
    
    ### Init player's hand
    def startGame(self, cards):
        self.cards = cards
    
    ### Add some cards to player's hand (if (s)he checked opponent's move, but (s)he was wrong)
    def takeCards(self, cards_to_take):
        self.cards = self.cards + cards_to_take

In [4]:
### Implement player's strategy. You can compare it with random player 
### (or some strategy implemented by one of you colleagues)
### Time limit per decision 0.01s !!!

class inf132326_inf132328(Player):
    
    def __init__(self, name):
        #(100,100) oznacza niewiadomą kartę
        self.opponentCards=[(100,100),(100,100),(100,100),(100,100),(100,100),(100,100),(100,100),(100,100)] 
        self.noneCard=True
        self.stack=[]
        self.Checked=False
        self.decision=2
        self.allCards=[]
        self.first=True
        self.cheating=False
        self.chosenCard=[]
    
    def putCard(self, declared_card):
        #jeżeli jest to pierwsza runda, to wszystkie karty, które mamy na ręcę zapamiętujemy, że są w grze
        if self.first:
            self.first=False
            self.allCards=self.cards.copy()      
        self.noneCard=False    
        
        #jeżeli możemy położyć dowolną kartę, nie jest to pierwsza tura i nie sprawdzaliśmy w tej turze, to znaczy, że przeciwnik wykonał akcję 'draw'
        #zatem z zapamiętanej listy kart ze stosu usuwamy 3 karty i dokładamy je do kart przeciwnika
        if not declared_card and len(self.stack)>0 and not self.Checked:
            n=min(len(self.stack),3)
            for i in range(n):    
                self.opponentCards.append(self.stack[len(self.stack)-1])
                self.stack.pop(len(self.stack)-1)
        self.Checked=False
        
        #wybieramy najniższą kartę z ręki
        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            #jeżeli nie możemy położyć tej karty, to pewnie będziemy kłamać...
            if card[0] < min_val: 
                i=0
                self.cheating=False
                p1=0.7 #prawdopodobieństwo skłamania
                #sprawdzamy, czy mamy na ręce kartę o tym samym nr, co leży na stosie
                #jeśli tak, to losujemy, czy skłamać kładąc naszą najniższą kartę jako tą kartę
                if (min_val,0) in self.cards:
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,0)
                elif (min_val,1) in self.cards:
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,1)
                elif (min_val,2) in self.cards:
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,2)
                elif (min_val,3) in self.cards: 
                    self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                    if self.cheating: self.chosenCard=(min_val,3)
                
                if not self.cheating:
                    #jeżeli nie skłamaliśmy, możemy skłamać kładąc kartę o 1 wyższą od karty na stosie, którą mamy na ręce
                    j=declared_card[0]+1
                    for f,c in self.cards:
                        if f==j:
                            if not self.cheating:
                                self.cheating=np.random.choice([True, False], p=[p1,1-p1])
                                self.chosenCard=(f,c)
                                
                #jeżeli nie skłamaliśmy:
                if not self.cheating:
                    self.chosenCard=self.opponentCards[0]
                    inAllCards=True
                    #losujemy kartę do 2 nr wyższą od obecnej na stosie, sprawdzamy, czy nie ma jej przeciwnik lub nie jest na stosie
                    #jeżeli znamy już wszystkie karty w grze, to sprawdzamy też, czy znajduje się wśród tych kart
                    while (self.chosenCard in self.opponentCards or self.chosenCard in self.stack or not inAllCards) and i<100: 
                        pom=random.randrange(4)
                        higher=random.randrange(3)
                        self.chosenCard=(min(min_val+higher,14),pom)
                        if len(self.allCards)==24:
                            inAllCards=False
                            if self.chosenCard in self.allCards:
                                inAllCards=True
                        i+=1
                    #losujemy, czy mamy w tym momencie skłamać
                    if i<100:
                        self.cheating=np.random.choice([True, False], p=[0.8,0.2])
                
                #deklarujemy wybraną kartę, że kładziemy na stół
                if self.cheating or i<100:
                    declaration = self.chosenCard

                #jeżeli nie kłamiemy, kładziemy najniższą możliwą kartę
                if not self.cheating:
                    mini=15
                    for f,c in self.cards:
                        if f>=declared_card[0] and f<mini:
                            declaration=(f,c)
                            card=(f,c)
                            mini=f
                
        self.stack.append(card)
        return card, declaration
    
    def checkCard(self, opponent_declaration):
        minCard = min(self.opponentCards, key=lambda x: x[0])
        
        #jeżeli przeciwnik kładzie kartę, którą podejrzewamy, że ma na ręce, nie będziemy sprawdzać
        if opponent_declaration in self.opponentCards:
            self.stack.append(opponent_declaration)
            self.opponentCards.remove(opponent_declaration)
            self.decision=0
        #jeżeli przeciwnik ma na ręce karty, których nie znamy
        elif (100,100) in self.opponentCards:
            self.stack.append((100,100))
            self.opponentCards.remove((100,100))
            self.decision=2
        #będziemy sprawdzać, bo przeciwnik kładzie kartę, której zapewne nie ma
        else:
            self.stack.append((100,100))
            self.opponentCards.remove(minCard)
            self.decision=1
        
        if opponent_declaration in self.cards: return True
        #sprawdzamy zawsze, kiedy zostanie nam na ręce 1 karta o nr niższym, niż ten na stosie
        #lepiej jest sprawdzić, niż brać 3 karty
        if len(self.cards) == 1 and self.cards[0][0] < opponent_declaration[0]: return True
        #jeżeli mamy 1 kartę, którą możemy położyć, to nie ma sensu sprawdzać
        if len(self.cards) == 1 and self.cards[0][0] >= opponent_declaration[0]: return False
        #sprawdzamy przeciwnika, jeśli nie kładł dowolnej karty, znamy przynajmniej 15 kart w grze i nie kładzie żadnej z nich
        if not self.noneCard and len(self.allCards)>=15 and opponent_declaration not in self.allCards: return True
        #jeśli przeciwnik kładł dowolną kartę, to sprawdzamy tylko jeśli karta ta ma nr 13 lub 14 i mamy co najmniej 3 karty na ręce
        if self.noneCard and opponent_declaration[0]<13: return False
        if self.noneCard and opponent_declaration[0]>=13: 
            if len(self.cards)<3: return False
            else: return True
        if self.decision==0: return False
        if self.decision==1: return True  
        #jeżeli znamy wszystkie karty przeciwnika i karta, którą kładzie ma nr>=najmniejszego nr na ręce, to nie sprawdzamy
        if (100, 100) not in self.opponentCards and minCard[0]>=opponent_declaration[0]: return False
        #jeśli podejrzewamy, że karta, którą położył przeciwnik już szła, to sprawdzamy
        if opponent_declaration in self.stack: return True
        
        #jeżeli mamy 2 karty na ręce, to niezbyt opłaca się sprawdzać, więc prawdopodobieństwo jest małe
        if len(self.cards)<3:
            return np.random.choice([True, False], p=[0.1, 0.9])
        #jeśli przeciwnik ma mało kart, to go sprawdzamy
        if len(self.opponentCards)<3:
            return np.random.choice([True, False], p=[0.8, 0.2])
        #jeżeli przeciwnik kładzie kartę o nr 9 lub 10, to raczej mówi prawdę
        if opponent_declaration[0]<11:
            return np.random.choice([True, False], p=[0.05, 0.95])
        #jeżeli wiemy, że przeciwnik ma na ręce kartę niższą od obecnej na stosie, to prawdopodobieństwo sprawdzenia rośnie
        if opponent_declaration[0]>=minCard[0]+4:
            return np.random.choice([True, False], p=[0.7, 0.3])
        if opponent_declaration[0]>=minCard[0]+3:
            return np.random.choice([True, False], p=[0.6, 0.4])
        if opponent_declaration[0]>=minCard[0]+2:
            return np.random.choice([True, False], p=[0.5, 0.5])
        #prawdopodobieństwo sprawdzenia jest tym większe, im więcej znamy kart w grze
        if opponent_declaration not in self.allCards:
            return np.random.choice([True, False], p=[len(self.allCards)/24, 1-len(self.allCards)/24])
        else:
            return np.random.choice([True, False], p=[0.2,0.8])
    
    #zapisujemy informacje o tym kto i jakie bierze karty, jakie karty sa usuwane ze stosu
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if checked:
            if not iDrewCards:
                if iChecked:
                    self.opponentCards.append(revealedCard)
                    self.stack.pop(len(self.stack)-1)
                    noTakenCards-=1
                    if revealedCard not in self.allCards:
                        self.allCards.append(revealedCard)
                for n in range(noTakenCards):
                    self.opponentCards.append(self.stack[len(self.stack)-1])
                    self.stack.pop(len(self.stack)-1)
            else:
                del self.stack[len(self.stack)-3 : len(self.stack)-1] 
            
            if iChecked:
                self.Checked=True
            else:
                self.noneCard=True
            
    def takeCards(self, cards_to_take):
        self.cards = self.cards + cards_to_take
        for c in cards_to_take:
            if c not in self.allCards:
                self.allCards.append(c)